In [1]:
import tensorflow as tf
from keras import callbacks
from keras import optimizers
from keras.datasets import cifar10
from keras.engine import Model
from keras.applications import xception as xception
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
batch = 250
num_epochs = 40
num_classes = 10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 37s 0us/step


In [3]:
import scipy as sp
import numpy as np
import pandas as pd
Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

X_train = np.array([sp.misc.imresize(x, 
                                     (72, 72)) for x in X_train])
X_test = np.array([sp.misc.imresize(x, 
                                    (72, 72)) for x in X_test])

/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [4]:
base_model = xception.Xception(weights='imagenet', 
                       include_top=False, 
                       input_shape=(72, 72, 3))

83689472/83683744 [==============================] - 10s 0us/step


In [0]:
# Original Model
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(10, activation='softmax')(x)

# Concatenating model and added layers
model = Model(base_model.input, predictions)

In [0]:
for layer in base_model.layers:
     layer.trainable = False

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 72, 72, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 35, 35, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 35, 35, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 35, 35, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [0]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=True)

In [0]:
train_datagen.fit(X_train)
train_generator = train_datagen.flow(X_train,
                                     Y_train, 
                                     batch_size=batch)

In [0]:
val_datagen = ImageDataGenerator(rescale=1. / 255,
    horizontal_flip=True)

val_datagen.fit(X_test)
val_generator = val_datagen.flow(X_test,
                                 Y_test,
                                 batch_size=batch)

In [0]:
train_steps_per_epoch = X_train.shape[0] // batch
val_steps_per_epoch = X_test.shape[0] // batch

history = model.fit_generator(train_generator,
                              steps_per_epoch=train_steps_per_epoch,
                              validation_data=val_generator,
                              validation_steps=val_steps_per_epoch,
                              epochs=1,
                              verbose=1)

Epoch 1/40
149/200 [=====================>........] - ETA: 13:07 - loss: 1.8722 - acc: 0.4427